# Fair Parameter Selection
This nootbook gives an overview of selecting parameters for the different models, such that they can be compared in a fair manner. 

In [ ]:
# number of components in the Tensor Train model
K_baseline = 10

# dimension of the data
M_DIM = 2

In [ ]:
K_MAX = 140
Ks = np.arange(2, K_MAX, 1)
Ms = np.arange(2, 3, 1)
params = np.zeros((3, len(Ms), len(Ks)))
legends = []

# compute number of parameters for each model
for i, M in enumerate(Ms):
    for j, K in enumerate(Ks):
        model_Train = m.TensorTrainGaussian(K, M)
        model_CP = m.CPGaussian(K, M)
        params[0, i, j] = model_Train.n_parameters()
        params[1, i, j] = model_CP.n_parameters()
        params[2, i, j] = K + M*K + K*M*M
    legends.append('TT, M='+str(M))
    legends.append('CP, M='+str(M))
    legends.append('GMM, M='+str(M))
    
# Make a vertical line
val = params[0, Ms==M_DIM, Ks==K_baseline][0]
line = val*np.ones(Ks.shape)
legends.append('Equality line')
    
f,ax = plt.subplots(figsize=(16, 5))
for i in range(len(Ms)):
    ax.plot(Ks, params[:,i].T, linewidth=3)
ax.plot(Ks, line, '--k', linewidth=3)
ax.legend(legends)
ax.set_xlabel('K of model')
ax.set_ylabel('Free parameters')
ax.grid('on')
ax.set_title('Number of free parameters vs. K for each model')
ax.set_ylim([0, 500])
ax.set_xlim([0, K_MAX])
plt.show()

In [ ]:
# Find indicies that achieve the same number of trainable parameters
idx_TT = np.argmax(params[0, Ms==M_DIM] >= val)
idx_CP = np.argmax(params[1, Ms==M_DIM] >= val)
idx_GMM = np.argmax(params[2, Ms==M_DIM] >= val)

# Number of components 
K_TT = Ks[idx_TT]
K_CP = Ks[idx_CP]
K_GMM = Ks[idx_GMM]

print('Selected number of components (K-values)')
print(f'TensorTrain : {K_TT}')
print(f'CP          : {K_CP}')
print(f'GMM(sklean) : {K_GMM}')

In [ ]:
# can also define a method for this, which is a lot faster 
# (especially needed for larger Ks)
# first do some faster computations for getting K's
def get_fair_Ks(K_tt, M, even=False):
    """ Computes fair parameters.
    Computes parameters for the GMM and CP models, 
    such that the three models can be compared fairly.

    Input
      K_tt  (int)   : The value of K for the Tensor Train model.
      M     (int)   : The number of dimensions of the data, 
                      which the models are trained on
      even  (bool)  : Whether the Ks are required to be even.

    Return
      K_tt  (int) : Same as input.
      K_gmm (int) : The value of K for the GMM model.
      K_cp  (int) : The value of K for the CP model.
    """

    if even:
      addi = 2
    else:
      addi = 1

    # number of free parameters for TT
    n_tt = m.TensorTrainGaussian(K_tt, M).n_parameters()

    # number of free parameters for GMM
    K_gmm = K_tt
    n_gmm = K_gmm * (1 + M + M*M)
    while n_gmm < n_tt:
        K_gmm += addi
        n_gmm = K_gmm * (1 + M + M*M)

    # number of free parameters for CP
    K_cp = K_gmm
    n_cp = m.CPGaussian(K_cp, M).n_parameters()
    while n_cp < n_tt:
        K_cp += addi
        n_cp = m.CPGaussian(K_cp, M).n_parameters()

    return K_tt, K_gmm, K_cp
  
  print(get_fair_Ks(K_baseline), M_DIM)